# Install libraries [optional]

In [ ]:
#Developed on Jupyter Notebook Data Science Stack (Docker)
#!pip install pmdarima
#!pip install tbats
#!conda install gcc
#!conda install -c conda-forge prophet

# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats,signal
from scipy.stats import chi2, chi2_contingency
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import statsmodels.api as sm

# Functions

## Plots

In [ ]:
def compute_loess_span(x_input,y_input, steps=0.02):
    import numpy as np
    from statsmodels.nonparametric.smoothers_lowess import lowess

    best_span = 0.51
    lowest_SSE = 10e20
    
    for i in np.arange(1,51,steps*100)/100:
        span=i
        y_pred = lowess(y_input,x_input, frac=span, it=5, return_sorted = False).T
        SSE = ((y_input-y_pred)**2).sum()
        if SSE<lowest_SSE:
            best_span = i
            lowest_SSE = SSE
    return (best_span,lowest_SSE)

In [ ]:
def plotTS(df, column, loess=True, regression=True, figsize=(16,4), title='', xlabel='', ylabel='', theme = 'darkgrid', points_alpha = 0.6, custom_ylim=None):
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    from sklearn.linear_model import LinearRegression
    from statsmodels.nonparametric.smoothers_lowess import lowess


    sns.set_style(theme)
    plt.figure(figsize=figsize)
    plt.scatter(x = df.index, y = df[column], marker='+', alpha=points_alpha, label='Data points')
    
    if loess==True:
        # model fitting
        span, sse = compute_loess_span(df.index,df[column])
        y_loess = lowess(df[column], df.index,  frac=span, it=5, return_sorted = False).T
        #plot
        plt.plot(df.index, y_loess, color='tomato', linewidth=2, label='LOESS (span={})'.format(span))
    if regression==True:
        # model fitting
        model = LinearRegression()
        x_regr = np.arange(len(df[column])).reshape(-1,1)
        model.fit(x_regr,df[column])
        coeff = model.coef_[0]
        y_predicted = model.predict(x_regr)
        #plot
        plt.plot(df.index, y_predicted, color='green', label='Regression line (coeff={})'.format(round(coeff,2)))
    if custom_ylim != None:
        plt.ylim(custom_ylim[0], custom_ylim[1])
    plt.xlim(df.index[0],df.index[-1])
    plt.title(title, fontsize=14)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()

In [ ]:
def scalelocationPlot(resid, title='', figsize=(16,8)):
    import pandas as pd
    import statsmodels.formula.api as smf
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    #lm
    resid = pd.DataFrame(resid, columns=['resid'])
    resid['step'] = range(len(resid))
    fit = smf.ols('resid ~ step', data=resid).fit()
    
    #get values
    model_fitted_y = fit.fittedvalues
    model_residuals = fit.resid
    model_norm_residuals = fit.get_influence().resid_studentized_internal
    model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))
    
    #plot
    fig = plt.figure(3)
    fig.set_figheight(figsize[1])
    fig.set_figwidth(figsize[0])
    plt.scatter(model_fitted_y, model_norm_residuals_abs_sqrt, alpha=0.5)
    sns.regplot(model_fitted_y, model_norm_residuals_abs_sqrt, scatter=False, ci=False, lowess=True,line_kws={'color': 'red', 'lw': 2, 'alpha': 0.8})
    fig.axes[0].set_title('Scale-Location {}'.format(title), fontsize=14)
    fig.axes[0].set_xlabel('Fitted values')
    fig.axes[0].set_ylabel('$\sqrt{|Standardized Residuals|}$')
    plt.show()

In [ ]:
def autocorrelationPlot(data, lags=30, title=''):
    from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
    import matplotlib.pyplot as plt
    
    fig = plot_acf(data, lags=lags, title='Auto-correlation for the first {} lags {}'.format(lags, title));
    fig.set_figwidth(16)
    fig.set_figheight(6)
    fig = plot_pacf(data, lags=lags, title='Partial Auto-correlation for the first {} lags {}'.format(lags, title));
    fig.set_figwidth(16)
    fig.set_figheight(6)
    plt.show()

In [ ]:
def qqPlot(data, title=''):
    fig = sm.qqplot(data, line ='45',fit=True,dist=stats.norm)
    fig.set_figheight(8)
    fig.set_figwidth(8)
    plt.title('Q-Q {}'.format(title))
    plt.show()

## Statistical tests

In [ ]:
def checkNormality(data, return_data=False):
    from scipy.stats import shapiro, jarque_bera, normaltest, skew, kurtosis
    
    #Shapiro-Wilk
    w, p_w = shapiro(data)
    print('*** Shapiro-Wilk Test ***')
    print('W: ', w)
    print('p-value: ' , p_w)
    if p_w > .05:
        print('Interpretation: the data was drawn from a normal distribution (Ho)')
    else:
        print('Interpretation: the data was not drawn from a normal distribution (Ha)')
    
    #Jarque-Bera
    jb, p_jb = jarque_bera(data)
    print('\n*** Jarque-Bera Test ***')
    print('Jarque-Bera JB: ', jb)
    print('p-value: ' , p_jb)
    if p_jb > .05:
        print('Interpretation: the data was drawn from a normal distribution (Ho)')
    else:
        print('Interpretation: the data was not drawn from a normal distribution (Ha)')
    
    #D’Agostino-Pearson
    k2, p_k2 = normaltest(data)
    print("\n*** D’Agostino-Pearson Test ***")
    print('k2: ', k2)
    print('p-value: ' , p_k2)
    if p_k2 > .05:
        print('Interpretation: the data was drawn from a normal distribution (Ho)')
    else:
        print('Interpretation: the data was not drawn from a normal distribution (Ha)')

    print('\n----------------------------------------------------------------------')
    print('Skewness: ', skew(data))
    print('Kurtosis : ', kurtosis(data))
    
    if return_data==True:
        return (w, p_w, jb, p_jb, k2, p_k2)

In [ ]:
def checkStationarity(data, kpss_type = 'ct', return_data=False, ci=.95):
    alpha = 1-ci
    
    if kpss_type in ['c','ct']:
        import warnings
        warnings.filterwarnings('ignore')
        from statsmodels.tsa.stattools import adfuller
        from statsmodels.tsa.stattools import kpss

        adf_test = adfuller(data,autolag='AIC')
        print('*** ADF Test ***')
        print('ADF Statistic: ', adf_test[0])
        print('p-value: ', adf_test[1])
        if adf_test[1] > alpha:
            print('Interpretation: The time series is non-stationary (Ho)')
        else:
            print('Interpretation: The time series is stationary (Ha)')

        kpss_test = kpss(data, regression=kpss_type, store=True)
        print('\n*** KPSS Test ***')
        print('KPSS Statistic:', kpss_test[0])
        print('p-value: ', kpss_test[1])

        if kpss_type == 'c':
            if kpss_test[1] > alpha:
                print('Interpretation: The time series is stationary (Ho)')
            else:
                print('Interpretation: The time series is not stationary (Ha)')
        if kpss_type == 'ct':
            if kpss_test[1] > alpha:
                print('Interpretation: The time series is trend stationary (Ho)')
            else:
                print('Interpretation: The time series is not trend stationary (Ha)')
    else:
        print('KPSS type shuld be c or ct!')
        print('c: checks if time series is stationary \nct: checks if time series is trend stationary')
    
    if return_data==True:
        return (adf_test[0],adf_test[1],kpss_test[0],kpss_test[1])

In [ ]:
def checkHomoscedasticity(resid, return_data=False, robust=False, includeWhite=False):
    import pandas as pd
    import statsmodels.formula.api as smf
    from statsmodels.compat import lzip
    import statsmodels.stats.api as sms
    from statsmodels.stats.diagnostic import het_white
    
    resid = pd.DataFrame(resid, columns=['resid'])
    resid['step'] = range(len(resid))
    fit = smf.ols('resid ~ step', data=resid).fit()
    
    if robust==False:
        #Bresuch-Pagan test
        lagrange_mult_statistic, p_value_bp, f_value, f_p_value = sms.het_breuschpagan(fit.resid, fit.model.exog, robust=False)
        print('*** Breusch-Pagan Test ***')
        print('Lagrange multiplier statistic: ', lagrange_mult_statistic)
        print('p-value: ', p_value_bp)
        if p_value_bp > .05:
            print('Interpretation: Homoscedasticity is present, the residuals are distributed with equal variance (Ho)')
        else:
            print('Interpretation: Heteroscedasticity is present, the residuals are not distributed with equal variance (Ha)')
    else:
        #Koenker test
        lagrange_mult_statistic, p_value_bp, f_value, f_p_value = sms.het_breuschpagan(fit.resid, fit.model.exog , robust=True)
        print('*** Koenker Test ***')
        print('Lagrange multiplier statistic: ', lagrange_mult_statistic)
        print('p-value: ', p_value_bp)
        if p_value_bp > .05:
            print('Interpretation: Homoscedasticity is present, the residuals are distributed with equal variance (Ho)')
        else:
            print('Interpretation: Heteroscedasticity is present, the residuals are not distributed with equal variance (Ha)')

    if includeWhite == True:    
        #White's test
        test_stat, p_value_w, f_stat, f_p_value = het_white(fit.resid,  fit.model.exog)
        print("\n*** White's test ***")
        print('Test statistic: ', test_stat)
        print('p-value: ', p_value_w)
        if p_value_w >.05:
            print('Interpretation: Homoscedasticity is present, residuals are equally scattered (Ho)')
        else:
            print('Interpretation: Heteroscedasticity is present, residuals are not equally scattered (Ha)')
        if return_data==True:
            return (lagrange_mult_statistic,p_value_bp,test_stat, p_value_w)
    else:
        if return_data==True:
            return (lagrange_mult_statistic,p_value_bp)

In [ ]:
def checkAutorcorrelation(resid, lags=30, return_data=False):
    import pandas as pd
    import statsmodels.formula.api as smf
    import statsmodels.api as sm
    import statsmodels.stats.diagnostic as dg
    
    #Ljung-Box
    lb_result = sm.stats.acorr_ljungbox(resid, lags=[lags], return_df=False)
    print("*** Ljung-Box Test [{} lags] ***".format(lags))
    print('Q: ', lb_result[0][0])
    print('p-value: ' , lb_result[1][0])
    if lb_result[1][0]>.05:
        print('Interpretation: the residuals are independently distributed (Ho)')
    else:
        print('Interpretation: The residuals are not independently distributed, they exhibit serial correlation (Ha)')
    
    #Breusch-Godfrey
    resid = pd.DataFrame(resid, columns=['resid'])
    resid['step'] = range(len(resid))
    fit = smf.ols('resid ~ step', data=resid).fit()
    bg_result = dg.acorr_breusch_godfrey(fit, nlags=lags)
    print('\n*** Breusch-Godfrey Test [{} lags] ***'.format(lags))
    print('Lagrange multiplier statistic: ', bg_result[0])
    print('p-value: ', bg_result[1])
    if bg_result[1]>.05:
        print('Interpretation: there is no autocorrelation at any order less than or equal to p (Ho)')
    else:
        print('Interpretation: There exists autocorrelation at some order less than or equal to p (Ha)')
    
    if return_data==True:
        return (lb_result[0][0], lb_result[1][0], bg_result[0], bg_result[1])

# Data loading

In [ ]:
sales_df = pd.read_csv('https://raw.githubusercontent.com/bolps/items_forecasting/main/Item_sales_ts.csv', sep=',', parse_dates=['date'])

In [ ]:
display(sales_df)
display(type(sales_df))
display(sales_df.dtypes)

In [ ]:
sales_df.isna().sum()

In [ ]:
sales_df.duplicated().sum()

There are no missing or duplicated values.

# Data manipulation

In [ ]:
sales_by_store = pd.pivot_table(data=sales_df, index='date', columns='store', values='sales', aggfunc='sum')
sales_by_store.columns = ['store_{}'.format(i) for i in range(1,11)]

In [ ]:
display(sales_by_store)
display(type(sales_by_store))
display(sales_by_store.dtypes)
display(sales_by_store.index)

In [ ]:
# Converting the index as date with daily frequency
sales_by_store.index = pd.to_datetime(sales_by_store.index)
sales_by_store = sales_by_store.asfreq('d')
display(sales_by_store.index)

# Data exploration

In order to unserstand and explore the charateristics of the dataset I defined some research questions regarding store performance, item preference and market in general.

## Analizing sales

>**Q1:** *Is market growing?*

>**Q2:** *Which are the best stores?*

>**Q3:** *Are sales between stores correlated?*

>**Q4:** *Which is the best month? And the best weekday?*

>**Q5:** *Which are the top 15 best selling items?*

>**Q6:** *Are top 15 items associated with specifc stores?*

### [Q1] *Is market growing?*

In [ ]:
sales_total = sales_by_store.sum(axis=1)
sales_total_by_year = sales_total.resample('Y').sum()
sales_total_by_year.index = sales_total_by_year.index.year
sales_total_by_year = sales_total_by_year/1000000
display(sales_total_by_year)

In [ ]:
plt.figure(figsize=(18,8))
sns.set_style('whitegrid')
sales_total_by_year.plot(kind='bar', rot=0, ylabel='Items sold (millions units)', xlabel='Year')
plt.title('Market growth \n total items sold - all stores', fontsize=14)
plt.show()

The market is growing, the number of items sold is increased every year from 2013 to 2017 reaching a peak of 10.75 millions unit sold.

*Note: As I am measuring the whole market performance i summed sales from every store/item*

### [Q2] *Which are the best stores?*

In [ ]:
fig = plt.figure(figsize=(18,10))
ax  = fig.add_subplot()
sns.set_style('whitegrid')
sns.boxplot(x="variable", y="value", data=pd.melt(sales_by_store), palette='Paired', width=0.65)
plt.title('Store benchmark', fontsize=14)
plt.xlabel('Store')
plt.ylabel('Items sold (unit/day)')
plt.show()

Thanks to the boxplot it is possible to appreciate that store 2 and store 8 are the best performers, while store 5, 5 and 7 are the worst.
It is also possible to observe that the variability increase as the sales goes up (this is a typicall of almost all sales data).

### [Q3] *Are sales between stores correlated?*

In [ ]:
cmap = sns.diverging_palette(230, 20, as_cmap=True)

mat_coeff = []
mat_pval = []
for i in list(range(len(sales_by_store.columns))):
    row_coeff = []
    row_pval = []
    for j in list(range(len(sales_by_store.columns))):
        temp = stats.pearsonr(sales_by_store.iloc[:,i],sales_by_store.iloc[:,j])
        row_coeff.append(temp[0])
        row_pval.append(temp[1])
    mat_coeff.append(row_coeff)
    mat_pval.append(row_pval)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.heatmap(mat_coeff, annot=True,  cmap=cmap,square=True,ax=axes[0],annot_kws={"size":10}, vmin=-1, vmax=1, xticklabels=sales_by_store.columns, yticklabels=sales_by_store.columns)
axes[0].set_title('r-value')
sns.heatmap(mat_pval, annot=True,  cmap=cmap,square=True,ax=axes[1],annot_kws={"size":10}, vmin=0, vmax=1, xticklabels=sales_by_store.columns, yticklabels=sales_by_store.columns)
axes[1].set_title('p-value')
fig.suptitle('Sales correlation between stores', fontsize=14)
plt.show()

Sales between stores shows an almost perfect correlation, this is really strange as in real life performance depends on many factors and close store can compete for the customers. This dataset could be synthetic.

### [Q4] *Which is the best month? And the best weekday?*

In [ ]:
sales_groupby = sales_df[['date','store','sales']].groupby(['date','store']).sum()
sales_groupby['month'] = (sales_groupby.index).to_series().map(lambda x : x[0].month)
sales_groupby['weekday'] = (sales_groupby.index).to_series().map(lambda x : x[0].isoweekday())
display(sales_groupby)

In [ ]:
fig = plt.figure(figsize=(18,10))
ax  = fig.add_subplot()
sns.set_style('whitegrid')
sns.boxplot(data=sales_groupby, x='month', y='sales', palette='Paired', width=0.65)
plt.title('Month benchmark', fontsize=14)
plt.ylabel('Items sold (unit/day)')
ax.set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,10))
ax  = fig.add_subplot()
sns.set_style('whitegrid')
sns.boxplot(data=sales_groupby, x='weekday', y='sales', palette='Paired', width=0.4)
plt.title('Weekday benchmark', fontsize=14)
plt.ylabel('Items sold (unit/day)')
ax.set_xticklabels(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])
plt.show()

Sales are better in summer, but november shows a positive peak. July is the best month and sales goes up from monday to sunday.

### [Q5] *Which are the top 15 best selling items?*

In [ ]:
sales_groupby_item = sales_df[['item','sales']].groupby(['item']).sum()
sales_groupby_item = sales_groupby_item.sort_values(by='sales', ascending=False)
sales_groupby_item['sales (millions units)'] = round(sales_groupby_item['sales']/1000000,3)
top_15_items = sales_groupby_item.head(15)
top_15_items.index = top_15_items.index.astype(str)
display(top_15_items)

In [ ]:
top_15_items = top_15_items.sort_values(by='sales', ascending=True)
plt.figure(figsize=(18,10))
sns.set_style('whitegrid')
plt.barh(top_15_items.index, top_15_items['sales (millions units)'], color= sns.color_palette("Paired"))
plt.title('Best sellers', fontsize=14)
plt.xlabel('Units sold (in millions)')
plt.ylabel('Product code')
plt.show()

Pruduct 15, 28 and 28 are the best sellers.

### [Q6] *Are top 15 items sales associated to specifc stores?*

In [ ]:
sales_store_item = pd.pivot_table(data=sales_df, index='item', columns='store', values='sales', aggfunc='sum')
sales_store_item.index = sales_store_item.index.astype(str)
sales_store_item.columns = sales_store_item.columns.astype(str)
sales_store_item_top15 = sales_store_item.loc[top_15_items.index]
display(sales_store_item_top15)

In [ ]:
stat, p, dof, expected = chi2_contingency(sales_store_item_top15)
print('dof=%d' % dof)
# interpret test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))

# interpret p-value
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

Sales of best selling items are equally distributed between stores, this again is really unlikely in real life (stores usally focus on selling something in particular).
This behavior is consistent if applied to all products.

# Focus on Time Series [Total items sold per store]

In [ ]:
descriptive_stat = sales_by_store.describe().T

Q1 = sales_by_store.quantile(0.25)
Q3 = sales_by_store.quantile(0.75)
IQR = Q3 - Q1

outlier_lower_count = (sales_by_store < (Q1 - 1.5 * IQR)).sum()
outlier_upper_count = (sales_by_store > (Q3 + 1.5 * IQR)).sum()
outlier_total_count = outlier_lower_count + outlier_upper_count

descriptive_stat['range']=descriptive_stat['max']-descriptive_stat['min']
descriptive_stat['IQR']=descriptive_stat['75%']-descriptive_stat['25%']
descriptive_stat['outlier_lower_count'] = outlier_lower_count
descriptive_stat['outlier_upper_count'] = outlier_upper_count
descriptive_stat['outlier_total_count'] = outlier_total_count

display(round(descriptive_stat,2))

Sales have different mean and standard deviation between stores, all stores have no outlier under the lower bound and just a few over the upper bound. This again is really unlikely in real life because there are good and bad days in terms of sales and also holidays (christmas, black friday, thanksgiving day).

In [ ]:
for store in sales_by_store.columns:
    store_number = store.split('_')[1]
    plotTS(sales_by_store, column=store, custom_ylim=(0,6000), title='Sales [Store {}]'.format(store_number), xlabel='Date', ylabel='Items sold (unit/day)')

Very high correlation (almost perfect) between sales in different stores and the same time pattern suggest that the data is likely to be synthetic. This 10 time series are kind of the same but with different trend and level.
I decide to focus on the study and the modelling of just one time serie. I choose store 2 because it shows the highest variability (an interesting aspect to study) that can be not so visible in other stores. 

# In depth analysis [Store 2]

## Graphical inspection

In [ ]:
store2_ts = pd.DataFrame(sales_by_store['store_2'])

In [ ]:
plotTS(store2_ts, column='store_2', custom_ylim=(0,6000), figsize=(16,4), title='Sales [Store 2]', xlabel='Date', ylabel='Items sold (unit/day)')

In [ ]:
fig = store2_ts[0:365].plot(figsize=(16,4), title='Sales 2013 [Store 2] - Item sales')
fig.axes.title.set_size(14)
plt.xlabel('Date')
plt.ylabel('Items sold (unit/day)')
plt.ylim(0,6000)
plt.show()

**Impressions from plots:**

From a graphical point of view, the time series I am studying shows some interesting characteristics that may be useful in defining the next steps. The time series has a non-stationary nature with a consistent increasing trend and at the same time it doesn't show breakpoints. There is graphical evidence of seasonality, in particular of both a weekly pattern and an annual cycle (also suggested by the boxplots of the exploratory analysis). It also seems to present the typical behavior that can be observed in most of the data relating to the sales of products in which as the number of units sold increases, so does the variance. If confirmed this might configure a situation where the trend is additive and seasonality multiplicative.

## Mathematical and statistical evidence

To test the graphical intuitions and gain other insights the following steps have been carried on:
1. Stationarity tests for having a statistical evidence of non stationarity, but also evidence of trend stationarity.
2. Power Spectral Density for seasonality (and cycle) investigation
3. Ljung-Box and Breusch-Godfrey test for statistical evidence of autocorrelation (alongiside Autocorrelation and Partial-Autocorrelation plots)
3. Decomposition and residuals analysis. In particular I will apply an additive model in order to verify, by studying the residuals, if the variance vary over time. If this is the case I will expect to find heteroscedasticity in the residuals (that will be tested with appropriate statistical tests).

### Stationarity

In [ ]:
checkStationarity(store2_ts['store_2'], kpss_type='c')

In [ ]:
checkStationarity(store2_ts['store_2'], kpss_type='ct')

### Power Spectral Density

In [ ]:
Fs = 1
f_per, Pxx_per = signal.periodogram(store2_ts['store_2'],Fs,window=None,return_onesided=True,scaling='density')
sns.set_style('darkgrid')
plt.figure(figsize=(16,6))
plt.title('Power Spectral Density [Store 2]', fontsize=14)
plt.plot(f_per[1:],Pxx_per[1:])
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD')

f_per = f_per[1:]
Pxx_per = Pxx_per[1:]
peaks = signal.find_peaks(Pxx_per[ f_per >= 0], prominence=np.quantile(Pxx_per, 0.99))[0]
peaks_freq = f_per[peaks]
peaks_power = Pxx_per[peaks]
plt.plot(peaks_freq, peaks_power, 'ro')
plt.xlim(0,0.5)
plt.show()

data = {'Freq':peaks_freq, 'Power':peaks_power}
df_PSD = pd.DataFrame(data)
df_PSD['Period (days)'] = 1/df_PSD['Freq']
display(df_PSD)

PSD confirmed the hypothesis of both an annual cycle (365 days) and a weekly seasonality (7 days).

### Autocorrelation

In [ ]:
autocorrelationPlot(store2_ts['store_2'], lags=30)
checkAutorcorrelation(store2_ts['store_2'].values, lags=30)

### Decomposition and analysis of residuals

#### Decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
#extracting weekly seasonality
sd_week = seasonal_decompose(store2_ts['store_2'], model='additive', period=7, two_sided=True)
#extracting yearly seasonality from time series adjusted by weekly seasonality
sd_year = seasonal_decompose(store2_ts['store_2'] -  sd_week.seasonal, model='additive', period=365, two_sided=True)

In [ ]:
f, axes = plt.subplots(4,1,figsize=(18,16))
plt.suptitle('Decomposition [Store 2]', y=0.92, fontsize=14)

#plotting trend component
axes[0].plot(sd_year.trend)
axes[0].set_title('Trend component', fontdict={'fontsize': 12})

#plotting weekly seasonal component
axes[1].plot(sd_week.seasonal)
axes[1].set_title('Weekly seasonal component', fontdict={'fontsize': 12})

#plotting yearly seasonal component
axes[2].plot(sd_year.seasonal)
axes[2].set_title('Yearly seasonal component', fontdict={'fontsize': 12})

#plotting residual of decomposition
axes[3].plot(sd_year.resid)
axes[3].set_title('Residual component', fontdict={'fontsize': 12})

for a in axes:
    a.set_ylabel('Items sold (units)')
    a.set_xlim(sd_year.trend.dropna().index[0], sd_year.trend.dropna().index[-1])

#showing chart
plt.show()

#### Decomposition residuals

In [ ]:
decomp_residuals = pd.DataFrame(sd_year.resid.dropna())
plotTS(decomp_residuals, column='resid', figsize=(16,4), title='Residual component [Store 2]')

##### Checking normality (residuals)

In [ ]:
qqPlot(decomp_residuals['resid'], title='Residual component [Store 2]')
checkNormality(decomp_residuals)

##### Checking stationarity (residuals)

In [ ]:
checkStationarity(decomp_residuals, kpss_type='c')

##### Checking homoscedasticity (residuals)

Since Breusch–Pagan test is sensitive to departures from normality (like the one in the residuals) I choose the Koenker test (studentized Breusch-Pagan).

More details in
> Daryanto, A. (2020). *Tutorial on Heteroskedasticity using HeteroskedasticityV3 SPSS macro.* The Quantitative Methods for Psychology, 16(5), 8-20.

In [ ]:
scalelocationPlot(decomp_residuals['resid'], figsize=(16,4))
checkHomoscedasticity(decomp_residuals, robust=True)

##### Checking autocorrelation (residuals)

In [ ]:
autocorrelationPlot(decomp_residuals['resid'], lags=30)
checkAutorcorrelation(decomp_residuals, lags=30)

Residuals suggest that the time serie is not heteroscedastic and so the seasonality is not multiplicative as appeared in the graphical inspection (at the same time it is possible that if the sales grow further it would be better to apply a multiplicative decomposition as there is a very light U shape in the scale location plot). The residual component is not white noise and there is an AR/MA process left.

Relevant considerations for next steps:
* It is not necessary to apply a Box-Cox trasformation
* Models should address trend
* Models should address weekly and yearly seasonality
* An AR/MA process is present

## Models comparison

The main goal of this section is to build a forecasting model that can effectively capture both annual and weekly patterns and provide a valid estimate of future values.

The following approaches will be tested in detail:
* **SARIMAX with Fourier Terms** as exogenous variables.
* **Double Seasonal Holt-Winters**
* **TBATS** (Exponential smoothing state space model with Box-Cox transformation, ARMA errors, Trend and Seasonal components)
* **Prophet** (Facebook), without holidays (as no holiday effect has been observed in data exploration)

The models will be *trained* on a training set containing sales from *2013-01-01 to 2017-10-31* and *tested* on the remaining actual data (*2017-11-01 to 2017-12-31*). 

Performance will be evaluated by using 
* **MSE**
* **MAE**
* **MAPE**

For each model residuals will be assessed in relation to normality, stationarity, autocorrelation and homoscedasticity.

The model with best performance and residuals will be applied to the remaining stores.

*Note: Holt-Winters will also be tested because I want to assess how well does this double-seasonality models perform compared to a 'basic' model with just one seasonal period.*

### Train-Test Split

In [ ]:
train = store2_ts['store_2'][:'2017-10-31']
test = store2_ts['store_2']['2017-11-01':]

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(train, color='blue', label='Train')
plt.plot(test, color='green', label='Test')
plt.xlim(train.index[0],test.index[-1])
plt.title('Train-Test Split [Store 2]', fontsize=14)
plt.legend()
plt.show()

### SARIMAX with Fourier Terms

#### Mathematical description

$ y_t = a + \sum_{k=1}^K \left[ \alpha_k\sin(2\pi kt/m) + \beta_k\cos(2\pi kt/m)\right] + N_t $

where $N_t$ is a SARIMA process and $K$ the number of Fourier terms.

#### Building the model

*Computing Fourier Terms*

Since the SARIMA model that I want to build allows me to take into account only one seasonality I have to add terms to the model in order to include the annual cycle. In this case the annual cycle is modeled through the addition of Fourier terms which are used as external regressors. 

In [ ]:
from pmdarima.preprocessing import FourierFeaturizer

In [ ]:
trans = FourierFeaturizer(365.25, 2)
y_prime, X = trans.fit_transform(store2_ts['store_2'])
X.index = store2_ts['store_2'].index
display(X)

Y_train = store2_ts['store_2'][:'2017-10-31']
X_train = X[:'2017-10-31']
Y_test = store2_ts['store_2']['2017-11-01':]
X_test = X['2017-11-01':]

*Finding the best model*

In [ ]:
from pmdarima import auto_arima

In [ ]:
arima_model = auto_arima(Y_train, start_p=1, start_q=1,
                         max_p=7, max_d=2, max_q=7, # maximum p d q
                         seasonal=True,   # Seasonality
                         m=7,              # Seasonality type (weekly)
                         max_P=7, max_D=2, max_Q=7, # maximum P D Q
                         suppress_warnings=True, 
                         n_jobs = -1,
                         maxiter = 50,
                         information_criterion = 'aic',
                         random_state=42,
                         trace=True,
                         exogenous=X_train)
print(arima_model.summary())

#### Checking the residuals

In [ ]:
arima_resid = arima_model.arima_res_.resid
plotTS(pd.DataFrame(arima_resid, columns=['resid']), column='resid', figsize=(16,4), title='SARIMAX Residuals [Store 2]')

##### Checking normality (SARIMAX residuals)

In [ ]:
qqPlot(arima_resid, title='- SARIMAX Residuals [Store 2]')
checkNormality(arima_resid)

##### Checking stationarity (SARIMAX residuals)

In [ ]:
checkStationarity(arima_resid, kpss_type='c')

##### Checking homoscedasticity (SARIMAX residuals)

In [ ]:
scalelocationPlot(arima_resid, figsize=(16,4))
checkHomoscedasticity(arima_resid, robust=True)

##### Checking autocorrelation (SARIMAX residuals)

In [ ]:
autocorrelationPlot(arima_resid, lags=30)
checkAutorcorrelation(arima_resid, lags=30)

#### SARIMAX Prediction

In [ ]:
y_pred_arima = arima_model.predict(n_periods=61, exogenous=X_test)
y_pred_arima = pd.Series(y_pred_arima)
y_pred_arima.index = test.index

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(test, color='blue', label='Ground truth')
plt.plot(y_pred_arima, color='red', label='Prediction')
plt.title('Forecast - SARIMAX with Fourier Terms [Store 2]', fontsize=14)
plt.xlim(test.index[0],test.index[-1])
plt.xlabel('Date')
plt.ylabel('Items sold (unit/day)')
plt.ylim(0,6000)
plt.legend()
plt.show()

In [ ]:
mse_sarimax = mean_squared_error(y_pred_arima,test.values)
mae_sarimax = mean_absolute_error(y_pred_arima,test.values)
mape_sarimax = mean_absolute_percentage_error(y_pred_arima,test.values)*100

print('SARIMAX model performance: ')
print('MSE: ', round(mse_sarimax,3))
print('MAE: ', round(mae_sarimax,3))
print('MAPE: ', round(mape_sarimax,3), '%')

### Double Seasonal Holt-Winters

#### Mathematical description

<img src="https://i.imgur.com/MBqLsPe.png" width="300">

You can find more details about the Double Seasonal Holt-Winters in:
> Hyndman, R., Koehler, A. B., Ord, J. K., & Snyder, R. D. (2008). *Forecasting with exponential smoothing: the state space approach.* Springer Science & Business Media.

#### Building the model

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
utils = importr('utils')
pandas2ri.activate()
time_series=robjects.r('ts')
forecast_package=importr('forecast')

In [ ]:
#converting the training data into an R time series object
r_times_series_data=time_series(train.values,frequency=1)

In [ ]:
dshw_model=forecast_package.dshw(r_times_series_data,364,7, h=61)
dshw_model_summary = dshw_model.rx('model')
print(dshw_model_summary)

#### Checking the residuals

In [ ]:
dshw_resid = np.array(dshw_model.rx('residuals'))
dshw_resid = pd.Series([x for x in dshw_resid[0]], index=train.index)
plotTS(pd.DataFrame(dshw_resid, columns=['resid']), column='resid', figsize=(16,4), title='Double Seasonal Holt-Winters Residuals [Store 2]')

##### Checking normality (Double Seasonal Holt-Winters residuals)

In [ ]:
qqPlot(dshw_resid, title='- Double Seasonal Holt-Winters Residuals [Store 2]')
checkNormality(dshw_resid)

##### Checking stationarity (Double Seasonal Holt-Winters residuals)

In [ ]:
checkStationarity(dshw_resid, kpss_type='c')

##### Checking homoscedasticity (Double Seasonal Holt-Winters residuals)

In [ ]:
scalelocationPlot(dshw_resid, figsize=(16,4))
checkHomoscedasticity(dshw_resid, robust=True)

##### Checking autocorrelation (Double Seasonal Holt-Winters residuals)

In [ ]:
autocorrelationPlot(dshw_resid, lags=30)
checkAutorcorrelation(dshw_resid, lags=30)

#### Double Seasonal Holt-Winters Prediction

In [ ]:
y_pred_dshw = np.array(dshw_model.rx('mean'))
y_pred_dshw = pd.Series([x for x in y_pred_dshw[0]], index =test.index)

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(test, color='blue', label='Ground truth')
plt.plot(y_pred_dshw, color='red', label='Prediction')
plt.title('Forecast - Double Seasonal Holt-Winters [Store 2]', fontsize=14)
plt.xlim(test.index[0],test.index[-1])
plt.xlabel('Date')
plt.ylabel('Items sold (unit/day)')
plt.ylim(0,6000)
plt.legend()
plt.show()

In [ ]:
mse_dshw = mean_squared_error(y_pred_dshw,test.values)
mae_dshw = mean_absolute_error(y_pred_dshw,test.values)
mape_dshw = mean_absolute_percentage_error(y_pred_dshw,test.values)*100

print('Double Seasonal Holt-Winters model performance: ')
print('MSE: ', round(mse_dshw,3))
print('MAE: ', round(mae_dshw,3))
print('MAPE: ', round(mape_dshw,3), '%')

### TBATS

#### Mathematical description

<img src="https://i.imgur.com/kY6IqVn.png" width="300">
<img src="https://yintingchou.com/posts/2017-05-03-bats-and-tbats-model/tbats2.png" width="300">

where $i = i_{th}$ seasonality. If it is double seasonality then  $i = 1,2$.

You can find more details about the TBATS in:
> De Livera, A. M., Hyndman, R. J., & Snyder, R. D. (2011). *Forecasting time series with complex seasonal patterns using exponential smoothing.* Journal of the American statistical association, 106(496), 1513-1527.

#### Building the model

In [ ]:
from tbats import TBATS

In [ ]:
tbats_model = TBATS(seasonal_periods=[7, 365.25], use_arma_errors=True)
tbats_fitted_model = tbats_model.fit(train.values)

In [ ]:
print(tbats_fitted_model.summary())

#### Checking the residuals

In [ ]:
tbats_resid = pd.Series(tbats_fitted_model.resid, index =train.index)
plotTS(pd.DataFrame(tbats_resid, columns=['resid']), column='resid', figsize=(16,4), title='TBATS Residuals [Store 2]')

##### Checking normality (TBATS residuals)

In [ ]:
qqPlot(tbats_resid, title='- TBATS Residuals [Store 2]')
checkNormality(tbats_resid)

##### Checking stationarity (TBATS residuals)

In [ ]:
checkStationarity(tbats_resid, kpss_type='c')

##### Checking homoscedasticity (TBATS residuals)

In [ ]:
scalelocationPlot(tbats_resid, figsize=(16,4))
checkHomoscedasticity(tbats_resid, robust=True)

##### Checking autocorrelation (TBATS residuals)

In [ ]:
autocorrelationPlot(tbats_resid, lags=30)
checkAutorcorrelation(tbats_resid, lags=30)

#### TBATS Prediction

In [ ]:
y_pred_tbats =  tbats_fitted_model.forecast(steps=61)
y_pred_tbats = pd.Series(y_pred_tbats, index=test.index)

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(test, color='blue', label='Ground truth')
plt.plot(y_pred_tbats, color='red', label='Prediction')
plt.title('Forecast - TBATS [Store 2]', fontsize=14)
plt.xlim(test.index[0],test.index[-1])
plt.xlabel('Date')
plt.ylabel('Items sold (unit/day)')
plt.ylim(0,6000)
plt.legend()
plt.show()

In [ ]:
mse_tbats = mean_squared_error(y_pred_tbats,test.values)
mae_tbats = mean_absolute_error(y_pred_tbats,test.values)
mape_tbats = mean_absolute_percentage_error(y_pred_tbats,test.values)*100

print('TBATS model performance: ')
print('MSE: ', round(mse_tbats,3))
print('MAE: ', round(mae_tbats,3))
print('MAPE: ', round(mape_tbats,3), '%')

### Prophet

#### Mathematical description

The math of Prophet is really complex and hard to summarize in a paper like this. You can find details about it in:
> Taylor SJ, Letham B. 2017. *Forecasting at scale*. PeerJ Preprints 5:e3190v2 https://doi.org/10.7287/peerj.preprints.3190v2

#### Building the model

In [ ]:
from prophet import Prophet

In [ ]:
train_prophet = pd.DataFrame({'y':train.values,'ds':train.index.to_list()})
prophet_model = Prophet(daily_seasonality=False, weekly_seasonality=True, yearly_seasonality=True)
prophet_fitted_model = prophet_model.fit(train_prophet)
future = prophet_fitted_model.make_future_dataframe(periods = 61)
forecast = prophet_fitted_model.predict(future)

#### Checking the residuals

In [ ]:
tmp = pd.merge(train_prophet, forecast.head(len(forecast)-61), on='ds')
prophet_resid = tmp['y'] - tmp['yhat']
plotTS(pd.DataFrame(prophet_resid, columns=['resid']), column='resid', figsize=(16,4), title='Prophet Residuals [Store 2]')

##### Checking normality (Prophet residuals)

In [ ]:
qqPlot(prophet_resid, title='- Prophet Residuals [Store 2]')
checkNormality(prophet_resid)

##### Checking stationarity (Prophet residuals)

In [ ]:
checkStationarity(prophet_resid, kpss_type='c')

##### Checking homoscedasticity (Prophet residuals)

In [ ]:
scalelocationPlot(prophet_resid, figsize=(16,4))
checkHomoscedasticity(prophet_resid, robust=True)

##### Checking autocorrelation (Prophet residuals)

In [ ]:
autocorrelationPlot(prophet_resid, lags=30)
checkAutorcorrelation(prophet_resid, lags=30)

#### Prophet Prediction

In [ ]:
y_pred_prophet = forecast['yhat'].tail(61)
y_pred_prophet.index = test.index

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(test, color='blue', label='Ground truth')
plt.plot(y_pred_prophet, color='red', label='Prediction')
plt.title('Forecast - Prophet [Store 2]', fontsize=14)
plt.xlim(test.index[0],test.index[-1])
plt.xlabel('Date')
plt.ylabel('Items sold (unit/day)')
plt.ylim(0,6000)
plt.legend()
plt.show()

In [ ]:
mse_prophet = mean_squared_error(y_pred_prophet,test.values)
mae_prophet = mean_absolute_error(y_pred_prophet,test.values)
mape_prophet = mean_absolute_percentage_error(y_pred_prophet,test.values)*100

print('Prophet model performance: ')
print('MSE: ', round(mse_prophet,3))
print('MAE: ', round(mae_prophet,3))
print('MAPE: ', round(mape_prophet,3), '%')

### Back to a basic model (Holt-Winters)

#### Mathematical description

$\begin{align*}
  \hat{y}_{t+h|t} &= \ell_{t} + hb_{t} + s_{t+h-m(k+1)} \\
  \ell_{t} &= \alpha(y_{t} - s_{t-m}) + (1 - \alpha)(\ell_{t-1} + b_{t-1})\\
  b_{t} &= \beta^*(\ell_{t} - \ell_{t-1}) + (1 - \beta^*)b_{t-1}\\
  s_{t} &= \gamma (y_{t}-\ell_{t-1}-b_{t-1}) + (1-\gamma)s_{t-m},
\end{align*}$

#### Building the model

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
holtwinters_fitted_model = ExponentialSmoothing(train.values, seasonal_periods=365, trend='add', seasonal='add',use_boxcox=False).fit()
print(holtwinters_fitted_model.params)

#### Checking the residuals

In [ ]:
holtwinters_resid = pd.Series(holtwinters_fitted_model.resid, index =train.index)
plotTS(pd.DataFrame(holtwinters_resid, columns=['resid']), column='resid', figsize=(16,4), title='Holt-Winters Residuals [Store 2]')

##### Checking normality (Holt-Winters residuals)

In [ ]:
qqPlot(holtwinters_resid, title='- Holt-Winters Residuals [Store 2]')
checkNormality(holtwinters_resid)

##### Checking stationarity (Holt-Winters residuals)

In [ ]:
checkStationarity(holtwinters_resid, kpss_type='c')

##### Checking homoscedasticity (Holt-Winters residuals)

In [ ]:
scalelocationPlot(holtwinters_resid, figsize=(16,4))
checkHomoscedasticity(holtwinters_resid, robust=True)

##### Checking autocorrelation (Holt-Winters residuals)

In [ ]:
autocorrelationPlot(holtwinters_resid, lags=30)
checkAutorcorrelation(holtwinters_resid, lags=30)

#### Holt-Winters Prediction

In [ ]:
y_pred_holtwinters = holtwinters_fitted_model.forecast(steps=61)
y_pred_holtwinters = pd.Series(y_pred_holtwinters, index=test.index)

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(test, color='blue', label='Ground truth')
plt.plot(y_pred_holtwinters, color='red', label='Prediction')
plt.title('Forecast - Holt-Winters [Store 2]', fontsize=14)
plt.xlim(test.index[0],test.index[-1])
plt.xlabel('Date')
plt.ylabel('Items sold (unit/day)')
plt.ylim(0,6000)
plt.legend()
plt.show()

In [ ]:
mse_holtwinters = mean_squared_error(y_pred_holtwinters,test.values)
mae_holtwinters = mean_absolute_error(y_pred_holtwinters,test.values)
mape_holtwinters = mean_absolute_percentage_error(y_pred_holtwinters,test.values)*100

print('Holt-Winters model performance: ')
print('MSE: ', round(mse_holtwinters,3))
print('MAE: ', round(mae_holtwinters,3))
print('MAPE: ', round(mape_holtwinters,3), '%')

### Summary

In [ ]:
f, axes = plt.subplots(5,1,figsize=(16,28))
plt.suptitle('Model comparison [Store 2]', y=0.92, fontsize=14)

axes[0].plot(y_pred_arima, color='red', label='SARIMA')
axes[0].plot(test, color='blue', label='Ground truth')
axes[0].set_title('SARIMAX vs Actual', fontdict={'fontsize': 12})

axes[1].plot(y_pred_dshw, color='orange', label='DSHW')
axes[1].plot(test, color='blue', label='Ground truth')
axes[1].set_title('Double Seasonal Holt-Winters vs Actual', fontdict={'fontsize': 12})

axes[2].plot(y_pred_tbats, color='green', label='TBATS')
axes[2].plot(test, color='blue', label='Ground truth')
axes[2].set_title('TBATS vs Actual', fontdict={'fontsize': 12})

axes[3].plot(y_pred_prophet, color='purple', label='Prophet')
axes[3].plot(test, color='blue', label='Ground truth')
axes[3].set_title('Prophet vs Actual', fontdict={'fontsize': 12})

axes[4].plot(y_pred_holtwinters, color='grey', label='Holt-Winters')
axes[4].plot(test, color='blue', label='Ground truth')
axes[4].set_title('Holt-Winters vs Actual', fontdict={'fontsize': 12})

for a in axes:
    a.set_ylabel('Items sold (units)')
    a.set_xlabel('Date')
    a.set_xlim(test.index[0], test.index[-1])
    a.set_ylim(0,6000)
    a.legend()

plt.show()

In [ ]:
data = {'Model':['SARIMA', 'DSHW', 'TBATS', 'PROPHET','HOLT-WINTERS'],
        'MSE':[mse_sarimax, mse_dshw, mse_tbats, mse_prophet,mse_holtwinters],
        'MAE':[mae_sarimax,mae_dshw,mae_tbats,mae_prophet,mae_holtwinters],
        'MAPE (%)':[mape_sarimax,mape_dshw,mape_tbats,mape_prophet,mape_holtwinters],
        'Stationarity (residuals)':['Stationary','Stationary','Stationary','Stationary','Not stationary'],
        'Distribution (residuals)':['Not normal','Not normal','Not normal','Not normal','Not normal'],
        'Homoscedasticity (residuals)':['Homoscedastic','Heteroscedastic','Heteroscedastic','Heteroscedastic','Heteroscedastic'],
        'Autocorrelation (residuals)':['Not autocorrelated','Not autocorrelated','Autocorrelated','Autocorrelated','Autocorrelated']}
 
all_models_summary = pd.DataFrame(data)
display(round(all_models_summary,2))

# Forecast [All Store]

In [ ]:
for store in sales_by_store.columns:
    #train-test split
    store_train = sales_by_store[store][:'2017-10-31']
    store_test = sales_by_store[store]['2017-11-01':]
    
    #model training and prediction
    store_train_data=time_series(store_train.values,frequency=1)
    store_dshw_model=forecast_package.dshw(store_train_data,364,7, h=61)
    store_y_pred_dshw = np.array(store_dshw_model.rx('mean'))
    store_y_pred_dshw = pd.Series([x for x in store_y_pred_dshw[0]], index=store_test.index)
    
    #prediction plot
    plt.figure(figsize=(16,4))
    plt.plot(store_test, color='blue', label='Ground truth')
    plt.plot(store_y_pred_dshw, color='red', label='Prediction')
    plt.title('Forecast - Double Seasonal Holt-Winters [{}]'.format(store), fontsize=14)
    plt.xlim(store_test.index[0],store_test.index[-1])
    plt.xlabel('Date')
    plt.ylabel('Items sold (unit/day)')
    plt.ylim(0,6000)
    plt.legend()
    plt.show()
    
    #model performance
    store_mse_dshw = mean_squared_error(store_y_pred_dshw,store_test.values)
    store_mae_dshw = mean_absolute_error(store_y_pred_dshw,store_test.values)
    store_mape_dshw = mean_absolute_percentage_error(store_y_pred_dshw,store_test.values)*100
    
    print('Double Seasonal Holt-Winters model performance [{}]: '.format(store))
    print('MSE: ', round(store_mse_dshw,3))
    print('MAE: ', round(store_mae_dshw,3))
    print('MAPE: ', round(store_mape_dshw,3), '%')